In [6]:
import pandas as pd

In [7]:
aqi_df = []
aqi_files_start_year = 2017

for i in range(5) :
    # Load the data into a dataframe
    df = pd.read_csv('raw_data/annual_aqi_by_county_' + str(i + aqi_files_start_year) + '.csv')
    #print(df.info())

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df[df['state'] == 'California'].drop(columns=['state'])
    
    aqi_df.append(df)

In [ ]:
# combine all aqi years dataframes into one
combined_aqi_df = pd.concat(aqi_df, ignore_index=True)
#print(combined_aqi_df)

In [ ]:
combined_aqi_df.to_csv('processed_data/combined_aqi_data.csv')

In [10]:
# clean asthma emergency department visits data

asthma_df = []
asthma_files_start_year = 2017

for i in range(5) :
    # load the data into a dataframe
    df = pd.read_excel('raw_data/Asthma_Emergency_' + str(i + asthma_files_start_year) + '.xlsx')
    #print(df.info())

    # clean up the dataframe
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.rename(columns={'counties': 'county'}, inplace=True)
    df = df[df['county'] != 'California'] # remove rows with 'California' in county name
    df = df.drop(columns=['lower_95%_limit', 'upper_95%_limit'])  # optional, but cleaner
 
    # add year column and reorder columns to have year after county (like in aqi_df)
    df['year'] = i + asthma_files_start_year
    columns_order = ['county', 'year'] + [col for col in df.columns if col not in ['county', 'year']]
    df = df[columns_order]

    asthma_df.append(df)


In [11]:
# combine all asthma years dataframes into one
combined_asthma_df = pd.concat(asthma_df, ignore_index=True)
#print(combined_asthma_df)

        county  year  age-adjusted_rate_per_10,000  number_of_cases
0      Alameda  2017                         56.04           8916.0
1       Alpine  2017                           NaN              NaN
2       Amador  2017                         56.02            175.0
3        Butte  2017                         45.29            935.0
4    Calaveras  2017                         54.51            197.0
..         ...   ...                           ...              ...
285     Tulare  2021                         26.95           1280.0
286   Tuolumne  2021                         23.59            104.0
287    Ventura  2021                         16.81           1312.0
288       Yolo  2021                         22.87            436.0
289       Yuba  2021                         29.34            235.0

[290 rows x 4 columns]


In [12]:
combined_asthma_df.to_csv('processed_data/combined_asthma_data.csv')

In [22]:
print(combined_asthma_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   county                        290 non-null    object 
 1   year                          290 non-null    int64  
 2   age-adjusted_rate_per_10,000  281 non-null    float64
 3   number_of_cases               281 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 9.2+ KB
None


In [20]:
# find counties missing

# count number of years county appears in aqi data
year_counts = combined_aqi_df.groupby('county')['year'].nunique()
#print(year_counts.sort_values())

#figure out which counties missing in aqi data
all_counties = combined_asthma_df['county'].unique()
aqi_counties = combined_aqi_df['county'].unique()
#print(len(all_counties), len(aqi_counties))

58 53


In [21]:
missing_counties = set(all_counties) - set(aqi_counties)
print("Counties missing in AQI data:", missing_counties)

Counties missing in AQI data: {'Alpine', 'Sierra', 'Modoc', 'Yuba', 'Lassen'}


In [26]:
for row in combined_asthma_df:
    if 'null' in row.number_of_cases :
        print(f"County {row.county} is missing in AQI data for year {row.year}")
    

AttributeError: 'str' object has no attribute 'number_of_cases'

In [39]:
for row in combined_asthma_df.itertuples():
    if 'NaN' in row:
        print(f"Missing data for county {row.county} in year {row.year}")

In [65]:
missing_counties_asthma = combined_asthma_df['number_of_cases'].isnull().sum()
print(missing_counties_asthma)

9


In [59]:
for index in combined_aqi_df.iloc[[-1]] :
    if index.isnull().any():
        print(index.county + 'is null')

AttributeError: 'str' object has no attribute 'isnull'

In [71]:
for index in combined_aqi_df.iterrows() :
    if index['number_of_cases'].value.isnull().any(axis=1):
        print(index.county + 'is null')

TypeError: tuple indices must be integers or slices, not str

In [66]:
# find rows with null values in ashtma data
missing_counties_asthma = combined_asthma_df[combined_asthma_df.isnull().any(axis=1)]

# print counties with null values
print(missing_counties_asthma['county'].unique())

# print the entire rows with null values
print(missing_counties_asthma)

Counties with null values:
['Alpine' 'Sierra']
Rows with null values:
     county  year  age-adjusted_rate_per_10,000  number_of_cases
1    Alpine  2017                           NaN              NaN
45   Sierra  2017                           NaN              NaN
59   Alpine  2018                           NaN              NaN
117  Alpine  2019                           NaN              NaN
161  Sierra  2019                           NaN              NaN
175  Alpine  2020                           NaN              NaN
219  Sierra  2020                           NaN              NaN
233  Alpine  2021                           NaN              NaN
277  Sierra  2021                           NaN              NaN


<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 1 to 277
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   county                        9 non-null      object 
 1   year                          9 non-null      int64  
 2   age-adjusted_rate_per_10,000  0 non-null      float64
 3   number_of_cases               0 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 360.0+ bytes


In [72]:
import itertools

years = sorted(combined_aqi_df['year'].unique())
skeleton = pd.DataFrame(itertools.product(all_counties, years),
                        columns=['county','year'])

merged = skeleton.merge(combined_aqi_df, on=['county','year'], how='left')

# Rows where median_aqi is missing:
gaps = merged[merged['median_aqi'].isna()]

# Count gaps per county
gap_counts = gaps.groupby('county').size().sort_values(ascending=False)
print(gap_counts)

county
Alpine    5
Lassen    5
Modoc     5
Sierra    5
Yuba      5
dtype: int64
